In [47]:
import os
import sys
from IPython import get_ipython
from pathlib import Path
import json
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import combinations
import traceback 

In [48]:
def get_script_directory():
    """
    Returns the ACTUAL directory containing the notebook/script.
    Works in:
    - VS Code Jupyter notebooks
    - Regular Jupyter Notebook/Lab
    - Standalone Python scripts
    """
    # If running in Jupyter
    if 'ipykernel' in sys.modules:
        try:
            # 1. First try VS Code's special attribute
            shell = get_ipython()
            if hasattr(shell, '__vsc_ipynb_file__'):
                return str(Path(shell.__vsc_ipynb_file__).parent)
            
            # 2. Try Jupyter notebook path (modern Jupyter)
            from notebook.notebookapp import list_running_servers
            servers = list_running_servers()
            if servers:
                import requests
                from urllib.parse import urljoin
                kernel_id = Path(get_ipython().config['IPKernelApp']['connection_file']).stem.replace('kernel-', '')
                for server in servers:
                    sessions = requests.get(urljoin(server['url'], 'api/sessions'), params={'token': server.get('token', '')}).json()
                    for session in sessions:
                        if session['kernel']['id'] == kernel_id:
                            return str(Path(server['notebook_dir']) / Path(session['notebook']['path']).parent)
            
            # 3. Fallback to current working directory
            return str(Path.cwd())
        except:
            return str(Path.cwd())
    
    # If running as a Python script
    return str(Path(__file__).parent.resolve())

In [49]:
# Function to delete all files and folders in path except .obsidian
def delete_all_except_obsidian(path):
    """
    Deletes all files and folders in the given path except for the .obsidian folder.
    """
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path) and item != '.obsidian':
            # Recursively delete the contents of the directory
            for root, dirs, files in os.walk(item_path, topdown=False):
                for file in files:
                    os.remove(os.path.join(root, file))
                for dir in dirs:
                    os.rmdir(os.path.join(root, dir))
            os.rmdir(item_path)  # Remove the now-empty directory
        elif os.path.isfile(item_path):
            os.remove(item_path)

In [50]:
def load_json(json_path):
    with open(json_path, 'r') as file:
        json_contents = json.load(file)
    return json_contents

In [51]:
def construct_url_dict(gid_dict, url_template):
    url_dict = {}
    for sheet, gid in gid_dict.items():
        full_url = url_template.replace("edit?gid=gid_value#gid=gid_value", f"export?format=csv&gid={gid}")
        url_dict[sheet] = full_url
    return url_dict

In [52]:
def construct_master_url_dict(sheets_dict):
    master_url_dict={}
    for spreadsheet, spreadsheet_dict in sheets_dict.items():
        spreadsheet_gid_dict=spreadsheet_dict['sheets']
        spreadsheet_url_template=spreadsheet_dict['link_template']
        spreadsheet_url_dict=construct_url_dict(spreadsheet_gid_dict, spreadsheet_url_template)
        master_url_dict[spreadsheet]=spreadsheet_url_dict
    return master_url_dict

In [53]:
def row_first_value(row):
    first_value = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    return first_value

def first_column_indecies(df):
    indecies_list = df.iloc[1:, 0]
    return indecies_list

In [54]:
def initial_content_dict_from_url(url):
    """
    Reads a CSV file and constructs a dictionary where each row's title (first column)
    is the key, and the rest of the row's data is a dictionary of key-value pairs.
    
    note: handles duplicate column names and row titles by grouping all rows with the same title 
    and adding a suffix to the column names.

    Args:
        url (str): The URL or path to the CSV file.

    Returns:
        dict: A dictionary with row titles as keys and row data as nested dictionaries.
    """
    df = pd.read_csv(url)
    
    # Convert float columns to Int64 where appropriate
    for col in df.select_dtypes(include=['float64']):
        if (df[col].dropna().apply(float.is_integer).all()):
            df[col] = df[col].astype('Int64')
    
    content_dict = {}
    
    for _, row in df.iterrows():
        if pd.notna(row.iloc[0]):  # First column as key
            index_value = row.iloc[0]
            
            if index_value not in content_dict:
                content_dict[index_value] = {}
            
            # Process each column
            for i, val in enumerate(row.iloc[1:]):
                if pd.notna(val):
                    original_col = df.columns[i+1]
                    base_col = original_col.split('.')[0]  # Remove pandas suffixes
                    
                    # Find next available column name
                    col_name = base_col
                    suffix = 1
                    while col_name in content_dict[index_value]:
                        suffix += 1
                        col_name = f"{base_col}_{suffix}"
                    
                    content_dict[index_value][col_name] = val
    
    return content_dict

In [55]:
def check_first_row_and_column_duplicates(df, detailed=False):
    """
    Enhanced duplicate checker for first column values and all headers.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        detailed (bool): If True, returns duplicate counts
        
    Returns:
        dict: {
            'column_duplicates': bool/Series,  # First column values
            'header_duplicates': bool/Series,  # All column headers
            'exact_header_duplicates': list   # List of duplicate header names
        }
    """
    results = {
        'column_duplicates': False,
        'header_duplicates': False,
        'exact_header_duplicates': []
    }
    
    # Check first column (values)
    first_col = df.iloc[:, 0]
    col_duplicates = first_col[first_col.duplicated(keep=False)]
    if detailed:
        results['column_duplicates'] = col_duplicates.value_counts().sort_values(ascending=False)
    else:
        results['column_duplicates'] = not col_duplicates.empty
    
    # Enhanced header check
    header_counts = pd.Series(df.columns).value_counts()
    dup_headers = header_counts[header_counts > 1]
    
    if not dup_headers.empty:
        results['header_duplicates'] = True
        results['exact_header_duplicates'] = dup_headers.index.tolist()
        if detailed:
            results['header_duplicates'] = dup_headers.sort_values(ascending=False)
    
    return results

In [56]:
def get_aliases(entry_data):
    """
    Extract and process aliases from specific columns in entry_data.

    Args:
        entry_data (dict): The dictionary containing row or sheet data to extract aliases.

    Returns:
        list: A list of processed aliases.
    """
    # Define the columns to check for aliases
    alias_columns = ['AKA', 'aliases', 'alias']

    # Check if any alias column exists in entry_data
    if not any(column in entry_data for column in alias_columns):
        return []  # Exit early if no alias columns are found

    aliases = set()

    # Extract and process aliases from the specified columns
    for column in alias_columns:
        if column in entry_data:
            # Split values by commas and strip whitespace
            raw_aliases = [alias.strip() for alias in str(entry_data[column]).split(',') if alias.strip()]
            for alias in raw_aliases:
                # Remove leading 'the' or 'a' (case-insensitive)
                alias = re.sub(r'^(the|a)\s+', '', alias, flags=re.IGNORECASE).strip()

                # Remove trailing 's or s
                if alias.endswith("'s"):
                    alias = alias[:-2]
                elif alias.endswith("s"):
                    alias = alias[:-1]

                aliases.add(alias)

    return list(aliases)

In [57]:
def search_keys_in_strings(search_keys, input_strings):
    """
    Checks if any of the search_keys exist as a string or substring in any of the input strings.

    Args:
        search_keys (list): List of search keys to check.
        input_strings (list): List of strings to search within.

    Returns:
        bool: True if any search key exists as a string or substring in any of the input strings, False otherwise.
    """
    for input_string in input_strings:
        for key in search_keys:
            if key in input_string:
                return True
    return False

In [58]:
def compare_and_update_references(dict_1, dict_2):
    """
    Compares search keys and content between two dictionaries and updates their references.

    Args:
        dict_1 (dict): The first dictionary with 'search_keys', 'content', and 'references'.
        dict_2 (dict): The second dictionary with 'search_keys', 'content', and 'references'.

    Returns:
        None: Updates the 'references' key in both dictionaries in place.
    """
    # Step 1: Extract search keys and convert to strings
    search_keys_1 = set(str(key).strip() for key in dict_1.get('search_keys', []))
    search_keys_2 = set(str(key).strip() for key in dict_2.get('search_keys', []))

    # Step 2: Check for overlap in search keys
    search_key_overlap = not search_keys_1.isdisjoint(search_keys_2)

    # Step 3: Extract strings to search in and handle lists or strings
    def extract_strings(content):
        if isinstance(content, list):
            return [str(value) for value in content]
        elif isinstance(content, str):
            return [content]
        return []

    strings_to_search_in_1 = extract_strings(dict_1.get('content', {}))
    strings_to_search_in_2 = extract_strings(dict_2.get('content', {}))

    # Step 4: Use search_keys_in_strings to compare content
    result_1 = search_keys_in_strings(search_keys_1, strings_to_search_in_2)
    result_2 = search_keys_in_strings(search_keys_2, strings_to_search_in_1)

    # Step 5: Combine conditions
    if search_key_overlap or result_1 or result_2:
        # Update references in dict_1
        references_1 = set(dict_1.get('references', []))
        references_1.add(dict_2.get('link', '').strip())
        dict_1['references'] = list(references_1)

        # Update references in dict_2
        references_2 = set(dict_2.get('references', []))
        references_2.add(dict_1.get('link', '').strip())
        dict_2['references'] = list(references_2)

In [59]:
def process_all_game_sheets(game_content_dict):
    """
    Cross-reference all entry_dicts across all games and sheets in game_content_dict.
    """
    all_entry_dicts = []
    for game, sheets in game_content_dict.items():
        for sheet, entries in sheets.items():
            for entry_key, entry in entries.items():
                # Ensure the entry itself is a dictionary
                if isinstance(dict_1.get('content', {}), dict) and isinstance(dict_2.get('content', {}), dict):
                    compare_and_update_references(dict_1, dict_2)
                # Add the entry to the list for comparison
                all_entry_dicts.append(entry)

    print(f"Total entries to compare: {len(all_entry_dicts)}")

    # Generate all pairs of entries for comparison
    entry_pairs = list(combinations(all_entry_dicts, 2))
    print(f"Total pairs to compare: {len(entry_pairs)}")

    for dict_1, dict_2 in entry_pairs:
        try:
            # Compare and update references only if 'content' is a dictionary
            compare_and_update_references(dict_1, dict_2)
        except Exception as e:
            print(f"Error comparing '{dict_1.get('link', 'Unknown')}' and '{dict_2.get('link', 'Unknown')}': {e}")

In [60]:
def dict_to_markdown(data, vault_path):
    """
    Convert nested dictionary structure to Markdown files.

    Args:
        data (dict): Nested dictionary in the format:
            {game1: {sheet1: {entry1: {'title': title, 'link': link, 
                    'content': {key1:value1, key2:value2,...}, 'search_keys': [...], 'references': [...]}}}, ...}
        vault_path (str): Root directory where files should be saved
    """
    for game, game_data in data.items():
        for sheet, sheet_data in game_data.items():
            for entry_key, entry_data in sheet_data.items():
                # Ensure the entry itself is a dictionary
                if not isinstance(entry_data, dict):
                    print(f"Warning: Entry data is not a dictionary in {game} -> {sheet} -> {entry_key}: {type(entry_data).__name__}")
                    continue

                # Process 'link' only if it's a string
                link = entry_data.get('link', '')
                if not isinstance(link, str):
                    print(f"Warning: 'link' is not a string in {game} -> {sheet} -> {entry_key}: {type(link).__name__}")
                    continue

                # Create the file path
                file_path = Path(vault_path) / f"{link}.md"
                file_path.parent.mkdir(parents=True, exist_ok=True)

                # Write the Markdown file
                with open(file_path, 'w', encoding='utf-8') as f:
                    # Process 'content' only if it's a dictionary
                    content = entry_data.get('content', {})
                    if isinstance(content, dict):
                        for key, value in content.items():
                            if isinstance(value, str) and '\n' in value:
                                value = value.replace('\n', '\n  ')
                            f.write(f"**{key}**: {value}\n\n")

                    # Process 'search_keys' only if it's a list or string
                    search_keys = entry_data.get('search_keys', [])
                    if isinstance(search_keys, (list, str)):
                        f.write(f"**search keys**: {', '.join(search_keys) if isinstance(search_keys, list) else search_keys}\n\n")

                    # Process 'references' only if it's a list
                    references = entry_data.get('references', [])
                    if isinstance(references, list):
                        f.write("\n## References\n")
                        for ref in references:
                            f.write(f"- [[{ref}]]\n")

In [61]:
def clean_square_brackets(text):
    """
    Escape single square brackets in the text to prevent them from being treated as links in Markdown.

    Args:
        text (str): The input text containing square brackets.

    Returns:
        str: The text with escaped square brackets.
    """
    return re.sub(r'(?<!\\)\[([^\]]+)\]', r'\\[\1\\]', text)

In [62]:
def clean_sprite_tags(text):
    """
    Remove <sprite name=...> tags from the text and ensure no double spaces.

    Args:
        text (str): The input text.

    Returns:
        str: The cleaned text.
    """
    # Remove <sprite name=...> tags
    text = re.sub(r'<sprite name=[^>]+>', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [63]:
def clean_sprite_tags_and_brackets(text):
    """
    Remove <sprite name=...> tags and escape square brackets in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The cleaned text.
    """
    text = clean_sprite_tags(text)  # Remove sprite tags
    return clean_square_brackets(text)  # Escape square brackets

In [64]:
def remove_empty_lines(s):
    """
    Remove all literal '/n' substrings and leading spaces from each line,
    and remove any empty (or whitespace-only) lines from the input string.

    Args:
        s (str): The input string.

    Returns:
        str: The cleaned string.
    """
    s = s.replace("/n", "")  # Remove literal '/n'
    lines = s.splitlines()  # Split into lines
    # Remove lines that are empty or contain only whitespace
    lines = [line.lstrip() for line in lines if line.strip()]
    return "\n".join(lines)  # Reassemble the string

In [65]:
def clean_filename(filename):
    """
    Cleans a filename by removing disallowed Windows characters and replacing them with underscores.

    Args:
        filename (str): The original filename string.

    Returns:
        str: The cleaned filename.
    """
    invalid = r'[<>:"/\\|?*]'
    filename = re.sub(r'^\s*(?:' + invalid + r'\s*)+', '', filename)  # Remove invalid chars at the start
    filename = re.sub(r'(?:\s*' + invalid + r')+\s*$', '', filename)  # Remove invalid chars at the end
    filename = re.sub(r'\s*(' + invalid + r')\s*', '_', filename)  # Replace invalid chars with underscores
    return remove_empty_lines(filename)  # Remove empty lines

In [66]:
def enhance_search_keys(search_keys, config):
    """
    Enhance search keys by:
    - Removing leading 'the', 'a', or 'an' and adding individual words for keys separated by delimiters.
    - Adding variations of words ending with 's' by removing the 's'.
    - Excluding specific words and digits from the enhanced keys.

    Args:
        search_keys (list): List of original search keys.
        config (dict): Configuration dictionary containing excluded words, digits, and delimiters.

    Returns:
        list: Enhanced list of search keys.
    """
    excluded_words = set(config.get("excluded_words", []))
    excluded_digits = set(config.get("excluded_digits", []))
    delimiters = "|".join(map(re.escape, config.get("delimiters", ["_", ":", "-", " "])))

    enhanced_keys = set()

    for key in search_keys:
        # Remove leading 'the', 'a', or 'an' (case-insensitive) and the separator after it
        key = re.sub(r'^(the|a|an)[{}]+'.format(delimiters), '', key, flags=re.IGNORECASE).strip()

        # Add the cleaned key to the enhanced keys
        enhanced_keys.add(key)

        # Split the key by delimiters
        parts = re.split(delimiters, key)
        for part in parts:
            if part and part.lower() not in excluded_words and part not in excluded_digits:
                enhanced_keys.add(part)

                # If the part ends with 's, add the version without 's
                if part.endswith("'s"):
                    enhanced_keys.add(part[:-2])
                # If the part ends with 's' (without the apostrophe), add the version without 's'
                elif part.endswith("s"):
                    enhanced_keys.add(part[:-1])

    return list(enhanced_keys)

In [67]:
def extract_sheet_content(url, sheet_name):
    """
    Reads a CSV file and constructs a content dictionary.
    Detects a row with the same name as the sheet title to extract aliases.

    Args:
        url (str): The URL or path to the CSV file.
        sheet_name (str): The name of the sheet.

    Returns:
        tuple: (content_dict, aliases)
    """
    df = pd.read_csv(url)
    print("Loaded DataFrame:")
    print(df)

    # Extract aliases and clean the DataFrame
    aliases = extract_aliases(df, sheet_name)

    # Construct the content dictionary
    content_dict = construct_content_dict(df)

    return content_dict, aliases


def extract_aliases(df, sheet_name):
    aliases = []
    print(f"Looking for aliases in sheet: {sheet_name}")
    print("First column values:", df.iloc[:, 0].values)
    
    if sheet_name in df.iloc[:, 0].values:
        aliases_row = df[df.iloc[:, 0] == sheet_name].iloc[0, 1:]
        aliases = [str(alias).strip() for alias in aliases_row if pd.notna(alias)]
        df.drop(df[df.iloc[:, 0] == sheet_name].index, inplace=True)
    
    print("Extracted aliases:", aliases)
    return aliases


def construct_content_dict(df):
    print("Constructing content dictionary...")
    print("DataFrame before processing:")
    print(df)
    
    content_dict = {}
    for _, row in df.iterrows():
        if pd.notna(row.iloc[0]):  # First column as key
            index_value = row.iloc[0]
            content_dict[index_value] = {}
            for i, val in enumerate(row.iloc[1:]):
                if pd.notna(val):
                    col_name = df.columns[i + 1]
                    content_dict[index_value][col_name] = val
    
    print("Constructed content dictionary:", content_dict)
    return content_dict

In [68]:
def construct_unified_dict(config_dict, master_url_dict):
    """
    Constructs a unified dictionary for both games and meta entries.

    Args:
        config_dict (dict): Configuration dictionary containing game and meta information.
        master_url_dict (dict): Dictionary containing URLs for each sheet.

    Returns:
        dict: The constructed unified dictionary.
    """
    unified_dict = {}

    for category in config_dict.get('games', []) + config_dict.get('meta', []):
        if category in master_url_dict:
            category_dict = {}
            for sheet_name, url in master_url_dict[category].items():
                try:
                    # Use the existing helper function to process the sheet
                    content, aliases = extract_sheet_content(url, sheet_name)
                    sheet_dict = {
                        'title': sheet_name,
                        'link': f"{'Meta Lore' if category in config_dict.get('meta', []) else category}/{sheet_name}",
                        'search_keys': [sheet_name] + aliases,
                        'references': [],
                        'content': content
                    }
                    category_dict[sheet_name] = sheet_dict
                except Exception as e:
                    print(f"Error processing sheet '{sheet_name}' in category '{category}': {e}")
                    # Create an empty sheet dictionary if there's an error
                    category_dict[sheet_name] = {
                        'title': sheet_name,
                        'link': f"{'Meta Lore' if category in config_dict.get('meta', []) else category}/{sheet_name}",
                        'search_keys': [sheet_name],
                        'references': [],
                        'content': {}
                    }
            unified_dict[category] = category_dict

    return unified_dict

In [69]:
def process_sheet(category, sheet_name, url):
    """
    Processes a single sheet and returns its content.

    Args:
        category (str): The category (game or meta).
        sheet_name (str): The name of the sheet.
        url (str): The URL of the sheet.

    Returns:
        tuple: (category, sheet_name, sheet_dict)
    """
    try:
        # Use the existing helper function to process the sheet
        raw_content = initial_content_dict_from_url(url)
        sheet_content = {}

        # Check if there's a row matching the sheet_name
        sheet_level_aliases = []
        if sheet_name in raw_content:
            # Extract aliases for the sheet-level search_keys
            sheet_level_aliases = get_aliases(raw_content[sheet_name])

        # Enrich each entry in the content
        for entry_name, entry_data in raw_content.items():
            # Clean the entry name for use as a key
            clean_entry_name = clean_filename(entry_name)

            # Extract aliases using get_aliases
            aliases = get_aliases(entry_data)

            # Construct the enriched entry structure
            enriched_entry = {
                'title': entry_name.strip(),
                'type': 'row',
                'link': f"{category}/{sheet_name}/{clean_entry_name}".strip(),
                'search_keys': [entry_name.strip()] + aliases,  # Add aliases to search_keys
                'references': [],
                'content': entry_data  # Keep the original key-value pairs
            }

            # Add the enriched entry to the sheet content
            sheet_content[entry_name] = enriched_entry

        # Construct the sheet dictionary
        sheet_dict = {
            'title': sheet_name,
            'type': 'sheet',
            'link': f"{category}/{sheet_name}",
            'search_keys': [sheet_name] + sheet_level_aliases,  # Add sheet-level aliases
            'references': [],
            'content': sheet_content,
        }

        return category, sheet_name, sheet_dict

    except Exception as e:
        # Log the error with traceback
        print(f"Error processing sheet '{sheet_name}' in category '{category}': {type(e).__name__}: {e}")
        print("Traceback:")
        traceback.print_exc()
        # Return an empty sheet dictionary in case of an error
        return category, sheet_name, {
            'title': sheet_name,
            'type': 'sheet',
            'link': f"{category}/{sheet_name}",
            'search_keys': [sheet_name],
            'references': [],
            'content': {}
        }

In [70]:
def construct_unified_dict(config_dict, master_url_dict):
    """
    Constructs a unified dictionary for both games and meta entries using multithreading.

    Args:
        config_dict (dict): Configuration dictionary containing game and meta information.
        master_url_dict (dict): Dictionary containing URLs for each sheet.

    Returns:
        dict: The constructed unified dictionary.
    """
    from concurrent.futures import ThreadPoolExecutor, as_completed

    unified_dict = {}

    # Use ThreadPoolExecutor to process sheets concurrently
    with ThreadPoolExecutor() as executor:
        futures = []

        # Submit tasks for each sheet in each category
        for category in config_dict.get('games', []) + config_dict.get('meta', []):
            if category in master_url_dict:
                for sheet_name, url in master_url_dict[category].items():
                    futures.append(executor.submit(process_sheet, category, sheet_name, url))

        # Collect results as they complete
        for future in as_completed(futures):
            category, sheet_name, sheet_dict = future.result()
            if category not in unified_dict:
                unified_dict[category] = {}
            unified_dict[category][sheet_name] = sheet_dict

    return unified_dict

In [71]:
# process_all_game_sheets(unified_dict)

In [73]:
get_script_directory()

'd:\\Weather Factory Lore\\Obsidian_mk_creation'

In [75]:
script_path = Path(get_script_directory())
# script_path = Path(__file__).parent
config_path = script_path / 'config.json'
sheets_path = script_path / 'sheets.json'
vault_path = script_path.parent / 'Obsidian Vault'

delete_all_except_obsidian(vault_path)
# Load configuration and sheets data
config_dict = load_json(config_path)
sheets_dict = load_json(sheets_path)

# Construct master URL dictionary
master_url_dict = construct_master_url_dict(sheets_dict)
unified_dict = construct_unified_dict(config_dict, master_url_dict)
# Construct game content dictionary
# game_content_dict = construct_game_content_dict(config_dict, master_url_dict)

# Construct meta lore dictionary
# meta_lore_dict = construct_initial_meta_dict(master_url_dict)

# Combine into master dictionary
# master_dict = game_content_dict
# master_dict['Meta Lore'] = meta_lore_dict

# Process all game sheets for cross-references
# process_all_game_sheets(master_dict)

# # Export to Markdown files
# dict_to_markdown(master_dict, vault_path)

print("Execution completed. Markdown files saved to:", vault_path)

Execution completed. Markdown files saved to: d:\Weather Factory Lore\Obsidian Vault


In [76]:
unified_dict

{'Cultist Simulator': {'General Influence': {'title': 'General Influence',
   'type': 'sheet',
   'link': 'Cultist Simulator/General Influence',
   'search_keys': ['General Influence'],
   'references': [],
   'content': {'Fleeting Reminiscence': {'title': 'Fleeting Reminiscence',
     'type': 'row',
     'link': 'Cultist Simulator/General Influence/Fleeting Reminiscence',
     'search_keys': ['Fleeting Reminiscence'],
     'references': [],
     'content': {'Moth': 2,
      'Secret Histories': 2,
      'Aspect 1': 'Memory',
      'Description': "A moment in time. In another moment, it'll be gone"}},
    'Glimmering': {'title': 'Glimmering',
     'type': 'row',
     'link': 'Cultist Simulator/General Influence/Glimmering',
     'search_keys': ['Glimmering'],
     'references': [],
     'content': {'Moth': 1,
      'Aspect 1': 'Advancement: A taste of what might be. [With enough of these cards, you can grow an Ability.]',
      'Description': "My emotions run higher than usual. There ar